In [1]:
import networkx as nx
import os
import random

# Asegura que las carpetas necesarias existen
def crear_estructura_directorios():
    base = "ARCHIVOS_REDES"
    subdirs = ["ER", "WS", "ER_parametros", "WS_parametros"]
    for sub in subdirs:
        os.makedirs(os.path.join(base, sub), exist_ok=True)

crear_estructura_directorios()

def contar_archivos_en_directorio(directorio):
    return len([f for f in os.listdir(directorio) if f.endswith(".txt")])


def guardar_lista_adyacencia(grafo, nombre_base, modelo, parametros):
    """
    Guarda el grafo en formato lista de adyacencia y los parámetros usados.

    Args:
        grafo (nx.Graph): Grafo a guardar.
        nombre_base (str): Nombre base para el archivo.
        modelo (str): 'ER' o 'WS'.
        parametros (dict): Parámetros a guardar en el archivo de parámetros.
    """
    carpeta_grafos = os.path.join("ARCHIVOS_REDES", modelo)
    carpeta_params = os.path.join("ARCHIVOS_REDES", f"{modelo}_parametros")

    i = contar_archivos_en_directorio(carpeta_grafos)
    nombre_grafo = f"{nombre_base}_{i}.txt"
    nombre_param = f"{nombre_base}_{i}.txt"

    # Guardar lista de adyacencia
    with open(os.path.join(carpeta_grafos, nombre_grafo), 'w') as f:
        for nodo in sorted(grafo.nodes()):
            vecinos = sorted(grafo.neighbors(nodo))
            for vecino in vecinos:
                f.write(f"{nodo} {vecino}\n")

    # Guardar parámetros
    with open(os.path.join(carpeta_params, nombre_param), 'w') as f:
        for clave, valor in parametros.items():
            f.write(f"{clave}: {valor}\n")


def generar_red_ER_conexa(N, k_prom, aceptancia_N, aceptancia_k, nombre_base="ER"):
    p = k_prom / (N - 1)

    while True:
        G = nx.erdos_renyi_graph(N, p)
        if not nx.is_connected(G):
            giant = max(nx.connected_components(G), key=len)
            G = G.subgraph(giant).copy()

        N_real = G.number_of_nodes()
        k_real = sum(dict(G.degree()).values()) / N_real

        if (abs(N_real - N) / N * 100 <= aceptancia_N and
                abs(k_real - k_prom) / k_prom * 100 <= aceptancia_k):
            parametros = {
                "N_ideal": N,
                "N_real": N_real,
                "k_prom_ideal": k_prom,
                "k_prom_real": round(k_real, 3)
            }
            guardar_lista_adyacencia(G, nombre_base, "ER", parametros)
            return G, N_real, k_real


def generar_red_WS(N, k, p, nombre_base="WS"):
    G = nx.watts_strogatz_graph(N, k, 0)
    feos = []
    guapos = []

    for edge in G.edges():
        if random.random() < p:
            feos.append(edge)
        else:
            guapos.append(edge)

    while feos:
        random.shuffle(feos)
        enlace0 = feos[0]
        nodo0_a, nodo0_b = enlace0
        encontrado = False

        for i in range(1, len(feos)):
            enlace1 = feos[i]
            nodo1_a, nodo1_b = enlace1
            if nodo0_a not in enlace1 and nodo0_b not in enlace1:
                nodos1 = [nodo1_a, nodo1_b]
                random.shuffle(nodos1)
                A = (nodo0_a, nodos1[0])
                B = (nodo0_b, nodos1[1])
                if A not in guapos and (A[1], A[0]) not in guapos and B not in guapos and (B[1], B[0]) not in guapos:
                    guapos.append(A)
                    guapos.append(B)
                    del feos[i]
                    del feos[0]
                    encontrado = True
                    break

        if not encontrado:
            guapos.append(feos[0])
            del feos[0]

    G_guapo = nx.Graph()
    G_guapo.add_nodes_from(range(N))
    G_guapo.add_edges_from(guapos)

    parametros = {
        "N": N,
        "k": k,
        "p": p
    }

    guardar_lista_adyacencia(G_guapo, nombre_base, "WS", parametros)
    return G_guapo


In [2]:
from datetime import datetime

def escribir_en_historial(nombre_archivo, parametros, archivo_inicio, archivo_fin, detalles_redes=[]):
    ruta = os.path.join("ARCHIVOS_REDES", nombre_archivo)
    with open(ruta, 'a', encoding='utf-8') as f:
        f.write("---------------------------------------------------------------------------------\n")
        f.write(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("Parámetros:\n")
        for clave, valor in parametros.items():
            f.write(f"{clave}: {valor}\n")
        f.write(f"Archivo inicial generado: {archivo_inicio}\n")
        f.write(f"Archivo final generado: {archivo_fin}\n")

        if detalles_redes:
            f.write("\nDetalles de las redes generadas:\n")
            for i, (nombre_archivo, N_real, k_real) in enumerate(detalles_redes):
                f.write(f"{i+1}. {nombre_archivo} -> N_real: {N_real}, k_real: {round(k_real, 3)}\n")

        f.write("---------------------------------------------------------------------------------\n\n")
        
def generar_varias_redes_ER(N_redes, N_nodos, k_prom, aceptancia_N, aceptancia_k, nombre_base="ER"):
    carpeta_grafos = os.path.join("ARCHIVOS_REDES", "ER")
    i_inicial = contar_archivos_en_directorio(carpeta_grafos)

    for _ in range(N_redes):
        generar_red_ER_conexa(N_nodos, k_prom, aceptancia_N, aceptancia_k, nombre_base)

    i_final = i_inicial + N_redes - 1
    archivo_inicio = f"{nombre_base}_{i_inicial}.txt"
    archivo_fin = f"{nombre_base}_{i_final}.txt"

    parametros_historial = {
        "N_redes": N_redes,
        "N_nodos": N_nodos,
        "k_prom": k_prom,
        "aceptancia_N (%)": aceptancia_N,
        "aceptancia_k (%)": aceptancia_k
    }

    escribir_en_historial("Historial_ER.txt", parametros_historial, archivo_inicio, archivo_fin)
    with open("ARCHIVOS_REDES/Historial_ER.txt", 'r', encoding='latin1') as f:
        contenido = f.read()

    with open("ARCHIVOS_REDES/Historial_ER..txt", 'w', encoding='utf-8') as f:
        f.write(contenido)



def generar_varias_redes_WS(N_redes, N_nodos, k, p, nombre_base="WS"):
    carpeta_grafos = os.path.join("ARCHIVOS_REDES", "WS")
    i_inicial = contar_archivos_en_directorio(carpeta_grafos)

    for _ in range(N_redes):
        generar_red_WS(N_nodos, k, p, nombre_base)

    i_final = i_inicial + N_redes - 1
    archivo_inicio = f"{nombre_base}_{i_inicial}.txt"
    archivo_fin = f"{nombre_base}_{i_final}.txt"

    parametros_historial = {
        "N_redes": N_redes,
        "N_nodos": N_nodos,
        "k": k,
        "p": p
    }

    escribir_en_historial("Historial_WS.txt", parametros_historial, archivo_inicio, archivo_fin)
    with open("ARCHIVOS_REDES/Historial_WS.txt", 'r', encoding='latin1') as f:
        contenido = f.read()
    with open("ARCHIVOS_REDES/Historial_WS.txt", 'w', encoding='utf-8') as f:
        f.write(contenido)


In [ ]:
## SOLO TOCAR LOS PARÁMETROS DESDE AQUÍ

N_redes=10
N_nodos=1000
k=4
p=0.3
generar_varias_redes_WS(N_redes, N_nodos, k, p)


In [3]:
## SOLO TOCAR LOS PARÁMETROS DESDE AQUÍ
N_redes = 1
N_nodos = 1000
k_inicial = 2
k_final = 8
delta_k = 0.1

k = k_inicial

while k < k_final:
    if k <= 2.3:
        aceptancia_N = 20
        aceptancia_k = 20
    elif 2.3 < k <= 2.5:
        aceptancia_N = 15
        aceptancia_k = 15
    elif 2.5 < k <= 3:
        aceptancia_N = 9
        aceptancia_k = 9
    elif 3 < k <= 3.5:
        aceptancia_N = 5
        aceptancia_k = 5
    elif 3.5 < k <= 4:
        aceptancia_N = 3
        aceptancia_k = 3
    elif 4 < k <= 4.5:
        aceptancia_N = 2
        aceptancia_k = 2
    else:  # k > 4.5
        aceptancia_N = 1
        aceptancia_k = 1

    generar_varias_redes_ER(N_redes, N_nodos, k, aceptancia_N, aceptancia_k)
    print (k)
    k += delta_k


2
2.1
2.2
2.3000000000000003
2.4000000000000004
2.5000000000000004
2.6000000000000005
2.7000000000000006
2.8000000000000007
2.900000000000001
3.000000000000001
3.100000000000001
3.200000000000001
3.300000000000001
3.4000000000000012
3.5000000000000013
3.6000000000000014
3.7000000000000015
3.8000000000000016
3.9000000000000017
4.000000000000002
4.100000000000001
4.200000000000001
4.300000000000001
4.4
4.5
4.6
4.699999999999999
4.799999999999999
4.899999999999999
4.999999999999998
5.099999999999998
5.1999999999999975
5.299999999999997
5.399999999999997
5.4999999999999964
5.599999999999996
5.699999999999996
5.799999999999995
5.899999999999995
5.999999999999995
6.099999999999994
6.199999999999994
6.299999999999994
6.399999999999993
6.499999999999993
6.5999999999999925
6.699999999999992
6.799999999999992
6.8999999999999915
6.999999999999991
7.099999999999991
7.19999999999999
7.29999999999999
7.39999999999999
7.499999999999989
7.599999999999989
7.699999999999989
7.799999999999988
7.899999999

In [ ]:
!git add .


In [ ]:
!git commit -m "nuevasER"


In [ ]:
!git pull

In [ ]:
\

In [ ]:
!git config --global user.name "DonIgnacioHerraiz"
!git config --global user.email "874462@unizar.es"


In [ ]:
!git config --list


In [ ]:
!git commit -m "Primer commit con redes y estructura"


In [ ]:
!git push
